In [1]:
import findspark
findspark.init()
# Spark:
from pyspark     import SparkConf
from pyspark     import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
# Python :
from datetime import datetime
import math
from pyspark.streaming import StreamingContext

listen_to_ip_caisse_1   = "54.37.12.49"
listen_to_port_caisse_1 = 12015

listen_to_ip_caisse_2   = "54.37.12.49"
listen_to_port_caisse_2 = 13000


current_date   = datetime.now().strftime("%Y-%m-%d %H:%M")
user           = "romain - exemple saprk streaming"
appName        = "%s le %s"%(user, current_date)
ip_fares       = "54.37.12.49"
master         = "spark://54.37.12.49:7077"
executor_cores = 2 
nb_cores_max   = 10
parallelism    = 4 * executor_cores
memory_by_node = "200M"
nb_executor    = nb_cores_max//executor_cores
dico_conf      = { "spark.app.name"             : appName        ,
                   "spark.master"               : master         ,
                   "spark.cores.max"            : nb_cores_max   , 
                   "spark.executor.cores"       : executor_cores ,
                   "spark.default.parallelism"  : parallelism    , 
                  #"spark.python.worker.memory" : memory_by_node ,
                   "spark.python.executor.memory" : memory_by_node }

conf = SparkConf()
for k,v in dico_conf.items():
    conf = conf.set(k,v)
sc = SparkContext.getOrCreate(conf=conf)

batch_duration = 1
ssc                  = StreamingContext(sc, batch_duration)
dstream_caisse_1     = ssc.socketTextStream(listen_to_ip_caisse_1, listen_to_port_caisse_1)
dstream_caisse_2     = ssc.socketTextStream(listen_to_ip_caisse_2, listen_to_port_caisse_2)
dstream = ssc.union(dstream_caisse_1, dstream_caisse_2)

In [2]:
ssc.checkpoint("hdfs://54.37.12.49:8020/spark_streaming_checkpoint/")

In [3]:
def update_achats(nouvelles_valeurs, valeur_actuelle ):
    if valeur_actuelle is None:
        valeur_actuelle = 0
    return sum(nouvelles_valeurs, valeur_actuelle)
        

In [5]:
data            = dstream.map(lambda x: x.split(","))
clients_facture = data.map(lambda x: (x[0], float(x[1])*float(x[2])))


In [6]:
# on veut savoir toutes les 5 secondes, ce qu'ils ont acheté pendant les 10 dernières secondes
limite = 3000
update_client   = clients_facture.reduceByKeyAndWindow(func           = lambda x,y : x+y, 
                                                       invFunc        = lambda x,y : x+y,
                                                       windowDuration = batch_duration*10, 
                                                       slideDuration  = batch_duration*5 )
client_avec_plus_de_3k_d_achat = update_client.filter(lambda x: x[1]>limite)
client_avec_plus_de_3k_d_achat.pprint()

# 9) activer l'écoute sur la socket

In [7]:
ssc.start()

-------------------------------------------
Time: 2018-02-26 15:35:48
-------------------------------------------
('client_c', 3028.0)

-------------------------------------------
Time: 2018-02-26 15:35:53
-------------------------------------------
('client_c', 8318.0)
('client_b', 3638.0)
('client_a', 4185.0)
('client_d', 4860.0)

-------------------------------------------
Time: 2018-02-26 15:35:58
-------------------------------------------
('client_c', 8900.0)
('client_b', 5224.0)
('client_a', 5664.0)
('client_d', 4558.0)



In [8]:
ssc.stop(False, True)